This notebook is for peak matching and pivoting the df from long to wide format for ML

**FIXME**: KS1341 is messed up. check this. Sent message to mary
    Temp Sol: remove KS1341 from data pool.

In [1]:
import pandas as pd

df_main1 = pd.read_csv('/Users/tylerstepaniak/Desktop/Jupyter Area/capstone-project/df_main1.csv')

# Checking for NAs and Nulls
print('CAS Null Count')
print(df_main1['CAS'].isnull().sum())
print('CAS NA Count')
print(df_main1['CAS'].isna().sum())
print('ID Null Count')
print(df_main1['ID'].isnull().sum())
print('ID NA Count')
print(df_main1['ID'].isna().sum())

CAS Null Count
0
CAS NA Count
0
ID Null Count
0
ID NA Count
0


In [2]:
# Group peaks by CAS and RT values
    # same CAS might show up at different RT so they should be separate features
    #  Column bleed, isomer, co-elutions
df_temp = df_main1.sort_values(by=['CAS', 'RT']).reset_index(drop=True)

# Tolerance level from literature
rt_tolerance = 0.2


# assign a peak group within each CAS
def rt_group(group):
    rt_groups = []
    current_group = 0
    last_rt = None

    for rt in group['RT']:
        if last_rt is None or abs((rt - last_rt)) > rt_tolerance:
            current_group += 1
        rt_groups.append(current_group)
        last_rt = rt

    group['RT_Group'] = rt_groups
    return group


df_temp = df_temp.groupby('CAS', group_keys=False).apply(rt_group)  # Apply grouping by CAS

# Create unique PeakID (CAS+RT)
df_temp['Feature_ID'] = df_temp['CAS'] + '_group' + df_temp['RT_Group'].astype(str)

# Keep representative RTs for reporting
    # you'll know the CAS, but it will just say group1 or group2 etc and you'll want to know the RT of that group
df_CAS_Group_LUT = df_temp.groupby(['Feature_ID', 'CAS'])['RT'].mean().reset_index()
df_CAS_Group_LUT.columns = ['Feature_ID', 'CAS', 'Mean_RT']

/var/folders/mn/x1lsf_0j0dl4sxlkkmh0q5kr0000gn/T/ipykernel_41105/51241056.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_temp = df_temp.groupby('CAS', group_keys=False).apply(rt_group)  # Apply grouping by CAS


In [12]:
# Pivot time
df_pivot = df_temp.pivot_table(
    index='PID',
    columns='Feature_ID',
    values='Area Pct',
    aggfunc='first'  # or 'mean' if duplicates
).reset_index()

# Merging Demographics back in
df_main2 = df_pivot.merge(
    df_main1[['PID', 'TB Status', 'Age', 'BMI', 'HIV Status']].drop_duplicates(),
    on='PID',
    how='left'
)

# Saving
df_main2.to_csv('df_main2.csv', index=False)
df_CAS_Group_LUT.to_csv('df_CAS_Group_LUT.csv', index=False)

In [17]:
# This is a QC Step

# Checking for dropped PIDs
print("Unique PIDs in df_pivot:", df_pivot['PID'].nunique(), "Should be 252")
print("Unique PIDs in demographics:", df_main1['PID'].nunique(), "Should equal the previous")

missing_pids = set(df_pivot['PID']) - set(df_main1['PID'])
print("PIDs in pivot not found in demographics:", missing_pids, "Should be an empty set")

print("\n" * 2, end="")

# Checking that merge went correctly with demographics
print(df_main2.columns)
df_main2.head()

Unique PIDs in df_pivot: 252 Should be 252
Unique PIDs in demographics: 252 Should equal the previous
PIDs in pivot not found in demographics: set() Should be an empty set


Index(['PID', '000035-27-8_group1', '000050-06-6_group1', '000050-70-4_group1',
       '000051-46-7_group1', '000051-73-0_group1', '000054-11-5_group1',
       '000054-49-9_group1', '000054-49-9_group2', '000055-21-0_group1',
       ...
       '959311-27-4_group1', '959311-27-4_group2', '959311-27-4_group3',
       '959311-27-4_group4', '959311-27-4_group5', '959311-27-4_group6',
       'TB Status', 'Age', 'BMI', 'HIV Status'],
      dtype='object', length=19457)


,PID,000035-27-8_group1,000050-06-6_group1,000050-70-4_group1,000051-46-7_group1,000051-73-0_group1,000054-11-5_group1,000054-49-9_group1,000054-49-9_group2,000055-21-0_group1,...,959311-27-4_group1,959311-27-4_group2,959311-27-4_group3,959311-27-4_group4,959311-27-4_group5,959311-27-4_group6,TB Status,Age,BMI,HIV Status
0,HC001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Negative,NaN,NaN,NaN
1,HC002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Negative,NaN,NaN,NaN
2,HC003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Negative,NaN,NaN,NaN
3,HC004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Negative,NaN,NaN,NaN
4,HC005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Negative,NaN,NaN,NaN


**Observations based on df_CAS_Group_LUT.csv**
* 19,452 unique Feature_IDs
    * matches the number of feature column s in pivot table
* 9,469 unique CAS numbers
    * each CAS has apx. 2 RT groups
        * Makes sense due to isomers, column bleed, etc.
* Mean_RT Values seem reasonable between min 0.1 and max 50
    * from my experience, this seems standard with GCMS
* No duplicated feature_IDs


In [ ]:
# Plotting number of RT groups per CAS for QC

import matplotlib.pyplot as plt


group_counts = df_CAS_Group_LUT.groupby('CAS')['Feature_ID'].nunique()

plt.figure(figsize=(10, 6))
plt.hist(group_counts, bins=30, edgecolor='black')
plt.xlabel('Number of RT Groups per CAS')
plt.ylabel('Number of CAS')
plt.title('Distribution of RT Groups per CAS')
plt.show()